# Предобработка текста

В данном ноутбуке представлена обновленная предобработка текста для обучения/инференса.

Этот вариант обработки включает в себя:
- удаление ссылок
- лемматизацию
- замену распространенных масок ругательств на базовые формы
- очистку пунктуации, которая определяет эмоциональную пунктуацию (например, смайлы и подряд идущие восклицательные знаки)
- исправление опечаток
- удаление стоп-слов

In [2]:
import gc
import re
import string
import pandas as pd
from pandarallel import pandarallel
from tqdm import tqdm
import spacy
from symspellpy import SymSpell, Verbosity

pandarallel.initialize(progress_bar=True)
tqdm.pandas()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
class TextProcessor:
    def __init__(self):
        self.mask_patterns = {
            re.compile(r'[f4$][u*µùúûü][c\(k<][k%]+', re.IGNORECASE): 'fuck',
            re.compile(r'[5$][h#][i1!][t+7]+', re.IGNORECASE): 'shit',
            re.compile(r'a[s$]{2}h[o0]l[e3]', re.IGNORECASE): 'asshole',
            re.compile(r'[b8][i1!][7+][c\(][h4]+', re.IGNORECASE): 'bitch',
            re.compile(r'\b[a@4][s$5]{2,}\b', re.IGNORECASE): 'ass',
            re.compile(r'\bp[1!i]ss\b', re.IGNORECASE): 'piss',
            re.compile(r'\b[d][a@4][m][nñ]+\b', re.IGNORECASE): 'damn',
            re.compile(r'\b[c\(][uµùúûü][nñ][t7+]+\b', re.IGNORECASE): 'cunt',
            re.compile(r'[p][0oöø][r][nñ]+', re.IGNORECASE): 'porn',
            re.compile(r'\b[w][h#][0oöø][r][e3€]+\b', re.IGNORECASE): 'whore',
            re.compile(r'\b[n][i1!][g6][g6][e3€][r®]+\b', re.IGNORECASE): 'nigger',
            re.compile(r'\b[f4$][a@4][g6][s$5]+\b', re.IGNORECASE): 'fags',
            re.compile(r'\b[s$5][l1!][uµùúûü][t7+]+\b', re.IGNORECASE): 'slut',
            re.compile(r'\b[d][1!][c\(][k%]+\b', re.IGNORECASE): 'dick',
            re.compile(r'\b[b8][0oöø][0oöø][b8]+\b', re.IGNORECASE): 'boob',
            re.compile(r'\b[c\(][0oöø][c\(][k%]+\b', re.IGNORECASE): 'cock',
            re.compile(r'\bb[o0]ll[o0]cks\b', re.IGNORECASE): 'bollocks',
            re.compile(r'\b[p][uµùúûü][s$5][s$5][y¥]+\b', re.IGNORECASE): 'pussy',
            re.compile(r'\b[m][0oöø][f4$][0oöø]+\b', re.IGNORECASE): 'mofo',
            re.compile(r'\b[t7+][w][a@4][t7+]+\b', re.IGNORECASE): 'twat',
            re.compile(r'\b[b8][a@4][s$5][t7+][a@4][r®][d]+\b', re.IGNORECASE): 'bastard',
            re.compile(r'\b[m][0oöø][t7+][h#][e3€][r®][f4$][uµùúûü][c$$][k%][e3€][r®]+\b', re.IGNORECASE): 'motherfucker',
            re.compile(r'\bw[a@]nk[e3]r\b', re.IGNORECASE): 'wanker'
        }

        self.swear_words = set(self.mask_patterns.values())
        self.emoticon_pattern = re.compile(
            r"([:=;]-?[)(\]\[DdPp/\\|]|<3|[!?]{2,})", 
            flags=re.VERBOSE|re.IGNORECASE
        )
        self.url_re = re.compile(r'(https?://\S+|www\.\S+)', re.IGNORECASE)
        
        self.nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
        self.stopwords = self.nlp.Defaults.stop_words

        self.sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
        self.sym_spell.load_dictionary(
            'frequency_dictionary_en_82_765.txt',
            encoding='utf-8-sig',
            term_index=0,
            count_index=1
        )
    
    def lemmatize_text(self, text: str):
        doc = self.nlp(text)
        processed_parts = []
        for token in doc:
            processed_parts.append(token.lemma_.lower())
            if token.whitespace_:
                processed_parts.append(' ')

        processed_text = ''.join(processed_parts)
        del processed_parts

        return processed_text
    
    def correct_swear_words(self, text: str):
        for pattern, base in self.mask_patterns.items():
            text = pattern.sub(base, text)
        return text
    
    def clear_punct(self, text: str):
        emoticons = self.emoticon_pattern.findall(text)
        keep_chars = set(''.join(emoticons))
        translator = str.maketrans(
            {char: ' ' for char in string.punctuation + '\n\t\r' if char not in keep_chars}
        )
        text = text.translate(translator)
        return re.sub(r'\s+', ' ', text).strip()
    
    def correct_typos(self, text: str):
        doc = self.nlp(text)
        tokens = []
        for token in doc:
            if not token.is_alpha:
                tokens.append(token.text)
                continue

            word = token.text
            suggestions = self.sym_spell.lookup(word, Verbosity.TOP)
            if suggestions and suggestions[0].term != word:
                word = suggestions[0].term
            if word not in self.stopwords:
                tokens.append(word)
        
        return ' '.join(tokens)
    
    def process_text(self, text: str):        
        # Удаление ссылок
        processed_text = self.url_re.sub('', text.lower())

        # Лемматизация текста
        processed_text = self.lemmatize_text(processed_text)
        
        # Замена замаскированных ругательств
        processed_text = self.correct_swear_words(processed_text)
        
        # Очистка пунктуации (кроме эмотиконов)
        processed_text = self.clear_punct(processed_text)

        # Исправление опечаток и удаление стоп-слов
        processed_text = self.correct_typos(processed_text)
        
        return processed_text

In [ ]:
total_rows = pd.read_csv('/Users/lev_k/HSE/YP/modified_train.csv', index_col=0).shape[0]
gc.collect()

processor = TextProcessor()
reader = pd.read_csv('/Users/lev_k/HSE/YP/modified_train.csv', chunksize=200000)
with tqdm(total=total_rows, desc="Processing") as pbar:
    for i, chunk in enumerate(reader):
        chunk['processed_ct'] = chunk['comment_text'].parallel_apply(processor.process_text)
        chunk.to_parquet(f'processed_chunk_{i}.parquet', index=False)

        pbar.update(len(chunk))

        del chunk
        gc.collect()

In [4]:
import glob

processed_df = pd.concat(
    [pd.read_parquet(f, engine='pyarrow') for f in glob.glob('processed_chunk_*.parquet')],
    ignore_index=False
)
processed_df.set_index('id', inplace=True)

In [16]:
processed_df.to_csv('preprocessed_train.csv')